<a href="https://colab.research.google.com/github/csukuangfj/colab/blob/master/sherpa_offline_recognition_python_api_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sherpa Offline Recognition Python API Demo

## Introduction

This notebook shows how to install [sherpa][sherpa] and use
it as offline recognizer.

Sherpa supports the models from icefall, the wenet framework and torchaudio.
- Conformer-CTC trained with gigaspeech
- Conformer-CTC trained with wenet framework
- torchaudio wav2vec 2.0 fine-tuned with CTC


[sherpa]: https//github.com/k2-fsa/sherpa

## Display the environment provided by Colab

In [ ]:
! python3 -m torch.utils.collect_env

PyTorch version: 2.0.1+cu118
Is debug build: False
CUDA used to build PyTorch: 11.8
ROCM used to build PyTorch: N/A

OS: Ubuntu 22.04.2 LTS (x86_64)
GCC version: (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Clang version: 14.0.0-1ubuntu1.1
CMake version: version 3.25.2
Libc version: glibc-2.35

Python version: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0] (64-bit runtime)
Python platform: Linux-5.15.109+-x86_64-with-glibc2.35
Is CUDA available: True
CUDA runtime version: 11.8.89
CUDA_MODULE_LOADING set to: LAZY
GPU models and configuration: GPU 0: Tesla T4
Nvidia driver version: 525.105.17
cuDNN version: Probably one of the following:
/usr/lib/x86_64-linux-gnu/libcudnn.so.8.9.0
/usr/lib/x86_64-linux-gnu/libcudnn_adv_infer.so.8.9.0
/usr/lib/x86_64-linux-gnu/libcudnn_adv_train.so.8.9.0
/usr/lib/x86_64-linux-gnu/libcudnn_cnn_infer.so.8.9.0
/usr/lib/x86_64-linux-gnu/libcudnn_cnn_train.so.8.9.0
/usr/lib/x86_64-linux-gnu/libcudnn_ops_infer.so.8.9.0
/usr/lib/x86_64-linux-gnu/libcudnn_ops_train.

It shows that the torch version is
`2.0.1+cu118`.

## Install k2

Please refer to
https://k2-fsa.github.io/k2/installation/from_wheels.html
for how to install `k2`.

In [ ]:
# Since the above output shows torch is 2.0.1+cu118, we install a version
# of k2 that is compiled against torch 2.0.1+cu118

! pip install k2==1.24.3.dev20230718+cuda11.8.torch2.0.1 -f https://k2-fsa.github.io/k2/cuda.html

Looking in links: https://k2-fsa.github.io/k2/cuda.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.9/117.9 MB 7.3 MB/s eta 0:00:00


## Install kaldifeat

Please refer to
https://csukuangfj.github.io/kaldifeat/installation/from_wheels.html
to install kaldifeat.

In [ ]:
! pip install kaldifeat==1.25.0.dev20230726+cuda11.8.torch2.0.1 -f https://csukuangfj.github.io/kaldifeat/cuda.html

Looking in links: https://csukuangfj.github.io/kaldifeat/cuda.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 574.0/574.0 kB 8.7 MB/s eta 0:00:00


## Install Sherpa

Please refer to
https://k2-fsa.github.io/sherpa/sherpa/install/from_wheels.html
to install sherpa.

In [ ]:
! pip install k2_sherpa==1.3.dev20230725+cuda11.8.torch2.0.1 -f https://k2-fsa.github.io/sherpa/cuda.html

Looking in links: https://k2-fsa.github.io/sherpa/cuda.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 18.6 MB/s eta 0:00:00


Check that sherpa has been install successfully:

In [ ]:
! python3 -c "import sherpa; print(sherpa.__version__)"

1.3.dev20230725+cuda11.8.torch2.0.1


In [ ]:
! sherpa-version

! sherpa-offline --help
! sherpa-online --help
! sherpa-online-microphone --help

! sherpa-offline-websocket-server --help
! sherpa-offline-websocket-client --help

! sherpa-online-websocket-server --help
! sherpa-online-websocket-client --help
! sherpa-online-websocket-client-microphone --help

sherpa version: 1.3
build type: Release
OS used to build sherpa: CentOS Linux release 7.9.2009 (Core)
sherpa git sha1: b7b3bb064a263520394fbf99f785597a6e688457
sherpa git date: Tue Jul 25 09:49:59 2023
sherpa git branch: release
PyTorch version used to build sherpa: 2.0.1+cu118
CUDA version: 11.8
cuDNN version: 
k2 version used to build sherpa: 1.24.3
k2 git sha1: e400fa3b456faf8afe0ee5bfe572946b4921a3db
k2 git date: Sat Jul 15 04:21:50 2023
k2 with cuda: ON
kaldifeat version used to build sherpa: 1.24
cmake version: 3.27.0
compiler ID: GNU
compiler: /opt/rh/devtoolset-9/root/usr/bin/c++
compiler version: 9.3.1
cmake CXX flags:    -Wall  -g  -D_GLIBCXX_USE_CXX11_ABI=0   -D_GLIBCXX_USE_CXX11_ABI=0 -Wno-unused-variable  -Wno-strict-overflow   -D_GLIBCXX_USE_CXX11_ABI=0
Python version: 3.10
[I] /var/www/sherpa/csrc/parse-options.cc:536:void sherpa::ParseOptions::PrintUsage(bool) const 2023-08-09 02:30:12.564 

Online (streaming) automatic speech recognition with sherpa.

Usage:
(1) View h

## Using CTC model as offline recognizer

In [ ]:
! sudo apt-get install -y git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


### Icefall

In [ ]:
# This model is trained using GigaSpeech
! GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/wgb14/icefall-asr-gigaspeech-conformer-ctc

Cloning into 'icefall-asr-gigaspeech-conformer-ctc'...
remote: Enumerating objects: 36, done.
remote: Total 36 (delta 0), reused 0 (delta 0), pack-reused 36
Unpacking objects: 100% (36/36), 1.09 MiB | 7.86 MiB/s, done.


In [ ]:
! cd icefall-asr-gigaspeech-conformer-ctc && \
  git lfs pull --include "exp/cpu_jit.pt" && \
  git lfs pull --include "data/lang_bpe_500/tokens.txt" && \
  git lfs pull --include "data/lang_bpe_500/HLG.pt"

In [ ]:
! cd icefall-asr-gigaspeech-conformer-ctc && \
  mkdir test_wavs && \
  cd test_wavs && \
  wget https://huggingface.co/csukuangfj/wav2vec2.0-torchaudio/resolve/main/test_wavs/1089-134686-0001.wav && \
  wget https://huggingface.co/csukuangfj/wav2vec2.0-torchaudio/resolve/main/test_wavs/1221-135766-0001.wav && \
  wget https://huggingface.co/csukuangfj/wav2vec2.0-torchaudio/resolve/main/test_wavs/1221-135766-0002.wav

--2023-08-09 02:32:21--  https://huggingface.co/csukuangfj/wav2vec2.0-torchaudio/resolve/main/test_wavs/1089-134686-0001.wav
Resolving huggingface.co (huggingface.co)... 65.8.49.24, 65.8.49.53, 65.8.49.2, ...
Connecting to huggingface.co (huggingface.co)|65.8.49.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 212044 (207K) [audio/wave]
Saving to: ‘1089-134686-0001.wav’

1089-134686-0001.wa 100%[===================>] 207.07K  --.-KB/s    in 0.05s   

2023-08-09 02:32:21 (3.94 MB/s) - ‘1089-134686-0001.wav’ saved [212044/212044]

--2023-08-09 02:32:21--  https://huggingface.co/csukuangfj/wav2vec2.0-torchaudio/resolve/main/test_wavs/1221-135766-0001.wav
Resolving huggingface.co (huggingface.co)... 65.8.49.24, 65.8.49.53, 65.8.49.2, ...
Connecting to huggingface.co (huggingface.co)|65.8.49.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 534924 (522K) [audio/wave]
Saving to: ‘1221-135766-0001.wav’

1221-135766-0001.wa 100%[==========

In [ ]:
# Decode with H
import os
os.chdir("/content/icefall-asr-gigaspeech-conformer-ctc")

from pathlib import Path
import sherpa

nn_model = "./exp/cpu_jit.pt"
tokens = "./data/lang_bpe_500/tokens.txt"
wave1 = "./test_wavs/1089-134686-0001.wav"
wave2 = "./test_wavs/1221-135766-0001.wav"
wave3 = "./test_wavs/1221-135766-0002.wav"

if not Path(nn_model).is_file():
  print("skipping test_icefall_ctc_model()")
  exit

print()
print("test_icefall_ctc_model()")

feat_config = sherpa.FeatureConfig()

feat_config.fbank_opts.frame_opts.samp_freq = 16000
feat_config.fbank_opts.mel_opts.num_bins = 80
feat_config.fbank_opts.frame_opts.dither = 0

config = sherpa.OfflineRecognizerConfig(
  nn_model=nn_model,
  tokens=tokens,
  use_gpu=False,
  feat_config=feat_config,
)

recognizer = sherpa.OfflineRecognizer(config)

s1 = recognizer.create_stream()
s2 = recognizer.create_stream()
s3 = recognizer.create_stream()

s1.accept_wave_file(wave1)
s2.accept_wave_file(wave2)
s3.accept_wave_file(wave2)

recognizer.decode_streams([s1, s2, s3])
print(s1.result)
print(s2.result)
print(s3.result)


test_icefall_ctc_model()
{"text":" AFTER EARLY NIGHTFALL THE YELLOW LAMPS WOULD LIGHT UP HERE AND THERE THE SQUALID QUARTER OF THE BROTHELS","timestamps":"[0.36,0.76,0.84,0.96,1.08,1.12,1.24,1.40,1.48,1.56,1.76,1.84,1.88,1.92,2.00,2.08,2.24,2.36,2.48,2.60,2.76,2.80,2.92,3.08,3.32,3.52,3.72,4.12,4.24,4.36,4.52,4.64,4.80,4.84,4.96,5.12,5.28,5.44,5.56,5.76,5.88,6.00]","tokens":[" AFTER"," E","AR","LY"," ","N","IGHT","F","AL","L"," THE"," ","Y","E","LL","OW"," LA","MP","S"," WOULD"," ","L","IGHT"," UP"," HERE"," AND"," THERE"," THE"," S","QU","AL","ID"," ","QU","AR","TER"," OF"," THE"," BRO","TH","EL","S"]}
{"text":" GOD AS A DIRECT CONSEQUENCE OF THE SIN WHICH MAN THUS PUNISHED HAD GIVEN HER A LOVELY CHILD WHOSE PLACE WAS ON THAT SAME DISHONOURED BOSOM TO CONNECT HER PARENT FOR EVER WITH THE RACE AND DESCENT OF MORTALS AND TO BE FINALLY A BLESED SOUL IN HEAVEN","timestamps":"[0.32,0.48,0.72,0.88,1.00,1.08,1.16,1.24,1.40,1.56,1.60,1.72,1.88,2.08,2.24,2.36,2.52,2.68,2.92,3.20,3.32,3.48,3.6

In [ ]:
# Decode with HLG
nn_model = "./exp/cpu_jit.pt"
tokens = "./data/lang_bpe_500/tokens.txt"
hlg = "./data/lang_bpe_500/HLG.pt"
wave1 = "./test_wavs/1089-134686-0001.wav"
wave2 = "./test_wavs/1221-135766-0001.wav"
wave3 = "./test_wavs/1221-135766-0002.wav"

if not Path(nn_model).is_file():
  print("skipping test_icefall_ctc_model_hlg_decoding()")
  exit

print()
print("test_icefall_ctc_model_hlg_decoding()")

feat_config = sherpa.FeatureConfig()

feat_config.fbank_opts.frame_opts.samp_freq = 16000
feat_config.fbank_opts.mel_opts.num_bins = 80
feat_config.fbank_opts.frame_opts.dither = 0

ctc_decoder_config = sherpa.OfflineCtcDecoderConfig(hlg=hlg)

config = sherpa.OfflineRecognizerConfig(
  nn_model=nn_model,
  tokens=tokens,
  feat_config=feat_config,
  ctc_decoder_config=ctc_decoder_config,
)

recognizer = sherpa.OfflineRecognizer(config)

s1 = recognizer.create_stream()
s2 = recognizer.create_stream()
s3 = recognizer.create_stream()

s1.accept_wave_file(wave1)
s2.accept_wave_file(wave2)
s3.accept_wave_file(wave2)

recognizer.decode_streams([s1, s2, s3])
print(s1.result)
print(s2.result)
print(s3.result)


test_icefall_ctc_model_hlg_decoding()
{"text":" AFTER EARLY NIGHTFALL THE YELLOW LAMPS WOULD LIGHT UP HERE AND THERE THE SQUALID QUARTER OF THE BROTHELS","timestamps":"[0.36,0.76,0.84,0.96,1.08,1.12,1.24,1.40,1.48,1.56,1.76,1.84,1.88,1.92,2.00,2.08,2.24,2.36,2.48,2.60,2.76,2.80,2.92,3.08,3.32,3.52,3.72,4.12,4.24,4.36,4.52,4.64,4.80,4.84,4.96,5.12,5.28,5.44,5.56,5.76,5.88,6.00]","tokens":[" AFTER"," E","AR","LY"," ","N","IGHT","F","AL","L"," THE"," ","Y","E","LL","OW"," LA","MP","S"," WOULD"," ","L","IGHT"," UP"," HERE"," AND"," THERE"," THE"," S","QU","AL","ID"," ","QU","AR","TER"," OF"," THE"," BRO","TH","EL","S"]}
{"text":" GOD AS A DIRECT CONSEQUENCE OF THE SIN WHICH MAN THUS PUNISHED HAD GIVEN HER A LOVELY CHILD WHOSE PLACE WAS ON THAT SAME DISHONOURED BOSOM TO CONNECT HER PARENT FOR EVER WITH THE RACE AND DESCENT OF MORTALS AND TO BE FINALLY A BLESED SOUL IN HEAVEN","timestamps":"[0.32,0.48,0.72,0.88,1.00,1.08,1.16,1.24,1.40,1.56,1.60,1.72,1.88,2.08,2.24,2.36,2.52,2.68,2.92,3.20,

### models from the wenet framework

#### English

In [ ]:
! cd /content && \
  GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/csukuangfj/wenet-english-model && \
  cd wenet-english-model  && \
  git lfs pull --include "final.zip"

Cloning into 'wenet-english-model'...
remote: Enumerating objects: 19, done.
remote: Total 19 (delta 0), reused 0 (delta 0), pack-reused 19
Unpacking objects: 100% (19/19), 752.53 KiB | 10.75 MiB/s, done.


In [ ]:
os.chdir("/content/wenet-english-model")

nn_model = "./final.zip"
tokens = "./units.txt"
wave1 = "./test_wavs/1089-134686-0001.wav"
wave2 = "./test_wavs/1221-135766-0001.wav"
wave3 = "./test_wavs/1221-135766-0002.wav"

if not Path(nn_model).is_file():
  print("skipping test_wenet_ctc_model()")
exit

print()
print("------test_wenet_ctc_model()------")

# models from wenet expect un-normalized audio samples
feat_config = sherpa.FeatureConfig(normalize_samples=False)

feat_config.fbank_opts.frame_opts.samp_freq = 16000
feat_config.fbank_opts.mel_opts.num_bins = 80
feat_config.fbank_opts.frame_opts.dither = 0

config = sherpa.OfflineRecognizerConfig(
    nn_model=nn_model,
    tokens=tokens,
    use_gpu=False,
    feat_config=feat_config,
)

recognizer = sherpa.OfflineRecognizer(config)

s1 = recognizer.create_stream()
s2 = recognizer.create_stream()
s3 = recognizer.create_stream()

s1.accept_wave_file(wave1)
s2.accept_wave_file(wave2)
s3.accept_wave_file(wave2)

recognizer.decode_streams([s1, s2, s3])
print(s1.result)
print(s2.result)
print(s3.result)


------test_wenet_ctc_model()------
{"text":" AFTER EARLY NIGHTFALL THE YELLOW LAMPS WOULD LIGHT UP HERE AND THERE THE SQUALID QUARTER OF THE BROTHELS","timestamps":"[0.60,0.96,1.38,1.62,1.74,1.98,2.10,2.40,2.64,2.82,3.00,3.24,3.48,3.72,3.90,4.32,4.50,4.62,4.74,4.86,5.10,5.46,5.58,5.76,5.94,6.12,6.18]","tokens":[" AFTER"," EARLY"," NIGHT","FA","LL"," THE"," YELLOW"," LAMP","S"," WOULD"," LIGHT"," UP"," HERE"," AND"," THERE"," THE"," S","QUA","LI","D"," QUARTER"," OF"," THE"," BRO","T","HEL","S"]}
{"text":" GOD AS A DIRECT CONSEQUENCE OF THE SIN WHICH MAN THUS PUNISHED HAD GIVEN HER A LOVELY CHILD WHOSE PLACE WAS ON THAT SAME DISHONORED BOSOM TO CONNECT HER PARENT FOR EVER WITH THE RACE AND DESCENT OF MORTALS AND TO BE FINALLY A BLESSED SOUL IN HEAVEN","timestamps":"[0.54,0.90,1.02,1.26,1.62,2.28,2.40,2.64,2.88,3.12,3.42,3.72,4.02,4.44,4.68,4.98,5.22,5.40,5.88,6.48,6.84,7.14,7.38,7.62,7.86,8.22,8.46,8.58,8.70,8.82,8.88,9.00,9.12,9.36,9.66,9.90,10.20,10.80,11.04,11.28,11.46,11.70,12.00,1

#### Chinese

In [ ]:
! cd /content && \
  GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/csukuangfj/wenet-chinese-model && \
  cd wenet-chinese-model && \
  git lfs pull --include "final.zip"

Cloning into 'wenet-chinese-model'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 16 (delta 0), reused 16 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), 697.35 KiB | 10.25 MiB/s, done.


In [ ]:
os.chdir("/content/wenet-chinese-model")

nn_model = "./final.zip"
tokens = "./units.txt"
wave1 = "./test_wavs/BAC009S0764W0121.wav"
wave2 = "./test_wavs/BAC009S0764W0122.wav"
wave3 = "./test_wavs/BAC009S0764W0123.wav"
wave4 = "./test_wavs/DEV_T0000000000.wav"
wave5 = "./test_wavs/DEV_T0000000001.wav"
wave6 = "./test_wavs/DEV_T0000000002.wav"

if not Path(nn_model).is_file():
  print("skipping test_wenet_ctc_model()")
exit

print()
print("------test_wenet_ctc_model()------")

# models from wenet expect un-normalized audio samples
feat_config = sherpa.FeatureConfig(normalize_samples=False)

feat_config.fbank_opts.frame_opts.samp_freq = 16000
feat_config.fbank_opts.mel_opts.num_bins = 80
feat_config.fbank_opts.frame_opts.dither = 0

config = sherpa.OfflineRecognizerConfig(
    nn_model=nn_model,
    tokens=tokens,
    use_gpu=False,
    feat_config=feat_config,
)

recognizer = sherpa.OfflineRecognizer(config)

s1 = recognizer.create_stream()
s2 = recognizer.create_stream()
s3 = recognizer.create_stream()
s4 = recognizer.create_stream()
s5 = recognizer.create_stream()
s6 = recognizer.create_stream()

s1.accept_wave_file(wave1)
s2.accept_wave_file(wave2)
s3.accept_wave_file(wave3)
s4.accept_wave_file(wave4)
s5.accept_wave_file(wave5)
s6.accept_wave_file(wave6)

recognizer.decode_streams([s1, s2, s3, s4, s5, s6])
print(s1.result)
print(s2.result)
print(s3.result)
print(s4.result)
print(s5.result)
print(s6.result)


------test_wenet_ctc_model()------
{"text":"甚至出现交易几乎停滞的情况","timestamps":"[0.76,1.00,1.24,1.48,1.80,2.04,2.28,2.48,2.72,2.92,3.12,3.36,3.56]","tokens":["甚","至","出","现","交","易","几","乎","停","滞","的","情","况"]}
{"text":"一二线城市虽然也处于调整中","timestamps":"[0.72,0.96,1.20,1.56,1.76,1.96,2.20,2.48,2.68,2.84,3.08,3.28,3.52]","tokens":["一","二","线","城","市","虽","然","也","处","于","调","整","中"]}
{"text":"但因为聚集了过多公共资源","timestamps":"[0.76,1.00,1.24,1.52,1.72,1.96,2.28,2.52,2.76,2.96,3.24,3.40]","tokens":["但","因","为","聚","集","了","过","多","公","共","资","源"]}
{"text":"对我做了介绍那么我想说的是大家如果对我的研究感兴趣呢","timestamps":"[0.48,0.64,0.76,0.88,1.04,1.20,1.96,2.08,2.28,2.40,2.52,2.64,2.76,3.40,3.52,3.68,3.76,3.88,3.96,4.04,4.16,4.28,4.40,4.60,4.76,4.96]","tokens":["对","我","做","了","介","绍","那","么","我","想","说","的","是","大","家","如","果","对","我","的","研","究","感","兴","趣","呢"]}
{"text":"重点呢想谈三个问题首先呢就是这一轮全球金融动荡的表现啊","timestamps":"[0.32,0.44,0.60,0.72,0.92,1.08,1.20,1.32,1.48,2.32,2.48,2.60,2.76,2.92,3.20,3.28,3.44,3.68,3.80,3.96,4.04,4.20,4

### torchaudio wav2vec 2.0

#### English

In [ ]:
# Note: There are other kinds of models fine-tuned with different
# amount of data. We use a model that is fine-tuned with 10 minutes of data.
! cd /content && \
  GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/csukuangfj/wav2vec2.0-torchaudio && \
  cd wav2vec2.0-torchaudio && \
  git lfs pull --include "wav2vec2_asr_base_10m.pt"

Cloning into 'wav2vec2.0-torchaudio'...
remote: Enumerating objects: 46, done.
remote: Total 46 (delta 0), reused 0 (delta 0), pack-reused 46
Unpacking objects: 100% (46/46), 1.27 MiB | 8.26 MiB/s, done.


In [ ]:
os.chdir("/content/wav2vec2.0-torchaudio")

nn_model = "./wav2vec2_asr_base_10m.pt"
tokens = "./tokens.txt"
wave1 = "./test_wavs/1089-134686-0001.wav"
wave2 = "./test_wavs/1221-135766-0001.wav"
wave3 = "./test_wavs/1221-135766-0002.wav"

if not Path(nn_model).is_file():
  print("skipping test_torchaudio_wav2vec2_0_ctc_model()")
exit

print()
print("test_torchaudio_wav2vec2_0_ctc_model()")

config = sherpa.OfflineRecognizerConfig(
    nn_model=nn_model,
    tokens=tokens,
    use_gpu=False,
)

recognizer = sherpa.OfflineRecognizer(config)

s1 = recognizer.create_stream()
s2 = recognizer.create_stream()
s3 = recognizer.create_stream()

s1.accept_wave_file(wave1)
s2.accept_wave_file(wave2)
s3.accept_wave_file(wave3)

recognizer.decode_streams([s1, s2, s3])
print(s1.result)
print(s2.result)
print(s3.result)


test_torchaudio_wav2vec2_0_ctc_model()
{"text":"E","timestamps":"[0.22]","tokens":["E"]}
{"text":"GAD AS A DERECT CONSEQUENCE OF THE SIN WHICH MAN THUS PUNISHED HAD GIVEN HERE A LOVELY CHILED WHOWS PLASE WAS ON THAT SAME DIS ANERED BOSOM TO CANECT HER PARENT FOR EVER WITHE RACSE AND DESENT OF MORTLES AND TO BE FINALY ABLESED SOL IN HEVEN ","timestamps":"[0.32,0.58,0.60,0.68,0.78,0.82,0.86,0.92,0.94,0.98,1.04,1.06,1.14,1.22,1.26,1.30,1.36,1.48,1.50,1.58,1.66,1.74,1.78,1.88,1.92,1.98,2.02,2.04,2.12,2.16,2.20,2.22,2.26,2.28,2.32,2.36,2.52,2.58,2.62,2.68,2.70,2.74,2.78,2.80,2.84,2.90,3.04,3.10,3.14,3.20,3.22,3.28,3.32,3.42,3.50,3.60,3.62,3.74,3.78,3.82,3.88,3.94,3.98,4.32,4.36,4.40,4.44,4.48,4.56,4.58,4.66,4.68,4.74,4.78,4.84,4.88,4.94,4.96,5.04,5.08,5.14,5.26,5.30,5.34,5.38,5.46,5.50,5.54,5.58,5.80,5.86,5.92,5.94,5.98,6.30,6.34,6.38,6.42,6.44,6.50,6.58,6.64,6.74,6.86,6.90,6.92,6.96,7.02,7.06,7.10,7.24,7.28,7.34,7.36,7.40,7.46,7.50,7.56,7.60,7.76,7.86,7.92,7.94,8.02,8.08,8.10,8.22,8.32,8.

#### German

In [ ]:
! git lfs pull --include "voxpopuli_asr_base_10k_de.pt"

In [ ]:
nn_model = "./wav2vec2_asr_base_10m.pt"
tokens = "./tokens.txt"
wave1 = "./test_wavs/20120315-0900-PLENARY-14-de_20120315.wav"
wave2 = "./test_wavs/20170517-0900-PLENARY-16-de_20170517.wav"

if not Path(nn_model).is_file():
  print("skipping test_torchaudio_wav2vec2_0_ctc_model()")
exit

print()
print("test_torchaudio_wav2vec2_0_ctc_model()")

config = sherpa.OfflineRecognizerConfig(
    nn_model=nn_model,
    tokens=tokens,
    use_gpu=False,
)

recognizer = sherpa.OfflineRecognizer(config)

s1 = recognizer.create_stream()
s2 = recognizer.create_stream()

s1.accept_wave_file(wave1)
s2.accept_wave_file(wave2)

recognizer.decode_streams([s1, s2])
print(s1.result)
print(s2.result)


test_torchaudio_wav2vec2_0_ctc_model()
{"text":"NADLISH GUPT ES EBWR GUFERE THIS HASCA HOWSES OR LOE SETE HITS A B ARTERS OR DESODIS ISETAIN THE VERSTAK MONDEGAN BARCIN AGLIHEINIAN STRAD IN DIGLAN RIRLRBEN ","timestamps":"[0.12,0.20,0.32,0.52,0.62,0.70,0.74,0.80,0.92,1.00,1.08,1.18,1.20,1.32,1.38,1.52,1.74,1.78,1.80,1.86,1.94,1.98,2.06,2.20,2.28,2.38,2.42,2.44,2.92,2.94,3.00,3.04,3.12,3.18,3.26,3.30,3.34,3.46,3.50,3.52,3.60,3.66,3.70,3.82,3.88,3.94,3.98,4.02,4.04,4.08,4.12,4.14,4.16,4.20,4.30,4.36,4.42,4.46,4.48,4.58,4.64,4.68,4.72,4.82,4.86,4.92,5.06,5.20,5.24,5.34,5.44,5.46,5.48,5.62,6.04,6.08,6.10,6.16,6.26,6.36,6.52,6.70,6.80,6.86,6.94,7.08,7.10,7.20,7.32,7.40,7.50,7.54,7.60,7.62,7.66,7.68,7.72,7.76,7.82,7.84,7.88,7.98,8.08,8.12,8.20,8.24,8.28,8.34,8.40,8.46,8.52,8.64,8.66,8.74,8.80,8.88,8.92,8.96,9.14,9.18,9.22,9.36,9.46,9.54,9.58,9.66,9.70,9.74,9.76,9.84,9.92,9.96,10.02,10.04,10.14,10.18,10.28,10.38,10.42,10.50,10.52,10.56,10.62,10.70,10.76,10.82,10.96,10.98,11.06,11.12,11.22,11